In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import regex as re
from tqdm import tqdm
tqdm.pandas()

In [2]:
arquivo ='D:/wiki_00'

In [3]:
articles = []

for i, line in enumerate(tqdm(open(arquivo, 'r'))):
    if i>=2000:
        break
    articles.append(json.loads(line))

2000it [00:00, 6053.26it/s]


In [4]:
df = pd.DataFrame(articles)

df = df.query("text != ''")

In [5]:
import regex as re
from tqdm import tqdm

regex_emoj = re.compile("[^"
    u"\U00000000-\U000000FF"
    u"\U0001F446-\U0001F450"
    u"\U0001F600-\U0001F64F"
                  "]+", re.UNICODE)

regex_espacos = re.compile(r'\s{2,}', re.UNICODE)

def process_text(text):
    text = re.sub(regex_emoj, ' ', text)
    text = re.sub(regex_espacos, ' ', text)
    return text

In [6]:
df["text"] = df["text"].progress_apply(process_text)

100%|████████████████████████████████████████████████████████████████████████████| 1663/1663 [00:00<00:00, 1997.60it/s]


In [7]:
df.drop(["id", "revid"], axis = 1, inplace = True)

In [8]:
df.head(2)

,url,title,text
0,https://pt.wikipedia.org/wiki?curid=220,Astronomia,Astronomia é uma ciência natural que estuda co...
1,https://pt.wikipedia.org/wiki?curid=223,América Latina,"A América Latina (em espanhol: ""América Latina..."


In [9]:
import sqlite3

conn = sqlite3.connect('textos.db')

In [10]:
df.to_sql(name='textos', con=conn)

1663

In [11]:
# pd.read_sql('select * from textos', conn)

In [12]:
conn.close()

In [13]:
import sqlite3

conn = sqlite3.connect('textos.db')

In [14]:
texto_buscado = "America"

In [15]:
df = pd.read_sql(f"""
select * from textos
where title LIKE '%{texto_buscado}%' or text LIKE '%{texto_buscado}%'
""", conn)

In [16]:
df.head()

,index,url,title,text
0,1,https://pt.wikipedia.org/wiki?curid=223,América Latina,"A América Latina (em espanhol: ""América Latina..."
1,5,https://pt.wikipedia.org/wiki?curid=229,Anarcocapitalismo,Anarcocapitalismo (também conhecido como anarq...
2,6,https://pt.wikipedia.org/wiki?curid=230,Anarquismo,"Nota: ""Anarquia"" redireciona para esta página..."
3,7,https://pt.wikipedia.org/wiki?curid=231,Albert Einstein,"Nota: Para outros significados, veja Albert E..."
4,8,https://pt.wikipedia.org/wiki?curid=232,Aquecimento global,Nota: Este artigo é sobre o aumento atual da ...
